# TPLC-Net 与 MOSSA 优化器的集成逻辑详解

本文档旨在详细解释在 `Module4_Demo_newshiyingdu_shixu.ipynb` 中实现的工作逻辑，即如何将 **TPLC-Net (深度学习时序预测模型)** 作为 **系统模型 (System Model / Digital Twin)** 嵌入到 **MOSSA (多目标优化算法)** 的适应度评估流程中。

## 1. 核心架构：模型预测控制 (MPC) 范式

我们的目标是寻找最优的 PID 参数（或设定点），使得未来的温室环境（温度、湿度）最符合番茄生长需求，同时能耗最低。

这是一个典型的 **模型预测控制 (Model Predictive Control, MPC)** 场景，其中：
1.  **控制器 (Controller)**: PID 加上优化算法 (MOSSA)。
2.  **系统模型 (System Model)**: TPLC-Net。它的作用是回答“如果我施加这个控制量，未来环境会变成什么样？”

### 逻辑流图
```mermaid
graph TD
    A[当前状态 State_t] --> B(MOSSA 优化器);
    B -->|生成候选解 Setpoints| C[PID 控制器];
    C -->|计算控制动作 u_t| D{TPLC-Net 预测器};
    
    subgraph "适应度评估循环 (horizon H)"
    D -->|输入: History + u_t| E[预测下一刻状态 State_t+1];
    E -->|更新 History| D;
    E --> F[计算 Yield & Cost];
    end
    
    F -->|反馈 Total Reward| B;
    B -->|迭代优化| B;
    B -->|输出最优解| G[执行控制];
```


## 2. 关键代码逻辑解析

在 `rolling_optimization_mossa` 函数中，我们实现了上述闭环。以下是关键步骤的详细拆解。

### 2.1 状态空间映射 (State Space Mapping)

**挑战**: 
*   **模拟环境**: 在简单的 Demo 中，我们只维护了核心变量 `(Temp, RH, CO2)`。
*   **TPLC-Net**: 训练时的输入是 `49` 维的全量温室传感器数据 (包括各种管道温度、天窗位置、光照传感器等)。

**解决方案**: `_map_to_features`
我们需要将 3 维的模拟状态映射到 49 维的模型输入。

```python
def _map_to_features(self, current_state, control_actions, external_weather):
    # 初始化 49 维向量 (全零或均值)
    features = np.zeros(self.input_dim)
    
    curr_t, curr_rh, curr_co2 = current_state
    u_temp, u_rh = control_actions
    
    # [关键] 填入当前模拟的核心状态
    features[10] = curr_t    # Tair (假设从数据集中得知的索引)
    features[9] = curr_rh    # Rhair
    features[2] = curr_co2   # CO2air
    
    # [关键] 填入控制量 (将 PID 输出映射到具体的执行机构)
    # 这是一种启发式映射：如果 PID 要求加热 (>0)，我们假设 PipeGrow 在工作；
    # 如果 PID 要求降温 (<0)，我们假设 VentLee (背风窗) 在开启。
    if u_temp > 0: features[7] = u_temp  # PipeGrow
    else: features[14] = abs(u_temp)     # VentLee
    
    # 注意：这里忽略了许多次要变量（如外部光照），在 Demo 中它们被置为 0 或均值。
    # 在生产环境中，应该尽可能填入真实的外部传感器数据。
    return features
```

### 2.2 自回归滚动预测 (Autoregressive Rolling Prediction)

这是 TPLC-Net 替代物理公式的核心。深度学习模型通常依赖**历史窗口 (Sequence Length)**。

*   **物理公式**: $State_{t+1} = f(State_t, u_t, External_t)$ (马尔可夫性，只看当前)
*   **TPLC-Net**: $State_{t+1} = NeuralNet(History_{t-L:t}, u_t)$ (依赖历史序列)

在 `objective_function` 的循环中：

```python
sim_history = None # 初始化历史为空 (冷启动)

# 滚动预测 H 步
for h in range(horizon):
    # 1. 传入当前状态、控制量、以及积累的 history
    result = predictor.predict_next_step(
        (curr_t, curr_r, curr_co2),
        (u_t_val, u_r_val),
        ...,
        history=sim_history # <--- 关键：传入历史
    )
    
    # 2. 获取预测结果 和 更新后的历史
    (next_t, next_r, next_co2), sim_history = result 
    
    # 3. 将 next_t 设为 curr_t，进入下一次循环
    curr_t = next_t
    # ...
```

**`sim_history` 的流动**:
1.  **Step 0**: `history` 为 None。预测器会把 `current_state` 复制 96 遍 (seq_len) 来伪造一个初始历史。
2.  **Step 0 -> 1**: 预测器输出 `next_state`，并将 `current_features` 追加到 history 末尾，通过 `np.roll` 挤掉最旧的数据。
3.  **Step 1 -> 2**: 使用更新后的 history 进行预测。

这确保了 TPLC-Net 始终能看到最近的模拟轨迹，从而捕捉趋势（例如升温速率的变化）。

### 2.3 物理公式 vs TPLC-Net: 为什么要替换？

| 特性 | 物理公式 (Simplified Physics) | TPLC-Net (Deep Learning) |
| :--- | :--- | :--- |
| **精度** | 低 (基于理想假设，忽略了热惯性、作物蒸腾的非线性) | **高** (从真实数据中学习了复杂的非线性交互) |
| **动态响应** | 瞬时响应 (只要加热，温度马上变) | **时序响应** (能模拟出加热后的滞后效应、惯性) |
| **计算量** | 极低 | 中等 (GPU 推理) |
| **部署** | 硬编码在代码中 | 需要加载 `.pt` 权重文件 |

**结论**: 将物理公式替换为 TPLC-Net，使得优化器 (MOSSA) 能基于**更真实的未来模拟**来评估 PID 参数，避免 PID 参数在仿真中表现完美但在现实中产生震荡（Sim-to-Real Gap）。

## 3. 部署注意事项

1.  **特征对齐**: 必须确保 `_map_to_features` 中的索引 (如 `features[10]`) 与训练 TPLC-Net 时的数据集列顺序严格一致。
2.  **归一化**: TPLC-Net 的权重是基于 `StandardScaler` 处理后的数据训练的。因此推理前必须**归一化** (使用训练集的 mean/std)，推理后必须**反归一化**。
3.  **冷启动**: 如果没有真实的历史数据（优化器刚启动时），使用 `np.tile` 复制当前状态是一种妥协。更好的做法是系统维护一个真实的滑动窗口 buffer，传入预测器。

## 4. 总结
你现在的集成逻辑是**正确**的：
1.  你没有抛弃 PID，而是用 TPLC-Net 来**评估** PID。
2.  你实现了**自回归**生成，符合时序模型的推理方式。
3.  你处理了**特征映射**，解决了维度不匹配的问题。

这正是“数字孪生驱动的优化控制”的标准实现范式。


In [ ]:
# 附：TPLCNetPredictor 精简代码回顾

class TPLCNetPredictor(GreenhousePredictor):
    def predict_next_step(self, current_state, control_actions, external_weather, history=None):
        # 1. 准备特征向量 (3维 -> 49维)
        features = self._map_to_features(current_state, control_actions, external_weather)
        
        # 2. 维护历史窗口 (Sequence Buffer)
        if history is None:
            # 冷启动：假设过去一直保持当前状态
            history = np.tile(features, (self.seq_len, 1))
        
        # 滑动窗口：移除最旧的，加入最新的
        new_history = np.roll(history, -1, axis=0)
        new_history[-1] = features
        
        # 3. 归一化 & 转张量
        x_np = (new_history - self.mean_vals) / self.std_vals
        x = torch.tensor(x_np).unsqueeze(0).to(self.device) 
        
        # 4. 模型推理
        with torch.no_grad():
            y_pred = self.model(x) 
            
        # 5. 提取预测值 & 反归一化
        # 假设模型输出顺序为 [Tair, Rhair, CO2air]
        pred_vals = y_pred[0, 0, :].cpu().numpy()
        
        next_t = pred_vals[0] * self.std_vals[10] + self.mean_vals[10]
        next_r = pred_vals[1] * self.std_vals[9] + self.mean_vals[9]
        next_co2 = pred_vals[2] * self.std_vals[2] + self.mean_vals[2]
        
        return (next_t, next_r, next_co2), new_history
